In [8]:
import refinitiv.data as rd
from refinitiv.data.content import symbol_conversion
from refinitiv.data.discovery import Chain
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
rd.open_session()


<refinitiv.data.session.Definition object at 0x15fe69550 {name='workspace'}>

In [9]:
cpn_dict = {'FXAN': 'FRB', 'FXDI': 'FRB', 'FXPP': 'FRB', 'FXPM': 'FRB', 'FXMF': 'FRB', 'FXZC': 'FRB', 'FXPV': 'FRB', 'FRBF': 'FRN', 'FRPV': 'FRN', 'FRFF': 'FRN', 'FXRV': 'FRN', 'FROT': 'FRN', 'FRFX': 'FRN',
            'FRVR': 'FRN', 'FRFZ': 'FRN', 'FRPM': 'FRN', 'FRSD': 'FRN', 'FRSU': 'FRN', 'VRFR': 'FRN', 'FRZF': 'FRN', 'RGOT': 'FRB', 'TBPD': 'FRN', 'FTZR': 'FRB', 'VRGR': 'FRB', 'ZRFX': 'FRB', 'ZRVR': 'FRB'}
irs_style = {'AUDSM6AB': 'AUD_SM6AB', 'AUDQM3AB': 'AUD_QM3AB', 'AEDAM3A': 'AED_AM3A', 'BHDAM2B': 'BHD_AM2B', 'CADSB3BA': 'CAD_SB3BA', 'CHFAB6L': 'CHF_AB6L', 'CNYQM7R': 'CNY_QM7R', 'CNYAM1D': 'CNY_AM1D', 'CZKAM6PR': 'CZK_AM6PR', 'DKKAB6C': 'DKK_AB6C', 'EURAB6E': 'EUR_AB6E', 'EURAB3E': 'EUR_AB3E', 'EURAM1E': 'EUR_AM1E', 'GBPSB6L': 'GBP_SB6L', 'HKDQM3H': 'HKD_QM3H', 'HUFAB6B': 'HUF_AB6B', 'IDRQM3J': 'IDR_QM3J', 'ILSAM3T': 'ILS_AM3T', 'INRSM6M': 'INR_SM6M', 'JPYSB6L': 'JPY_SB6L', 'KRWQMCD': 'KRW_QMCD', 'KWDAM3K': 'KWD_AM3K',
             'MYRQB3KL': 'MYR_QB3KL', 'NOKAB6O': 'NOK_AB6O', 'NZDSM3NB': 'NZD_SM3NB', 'PHPQB3P': 'PHP_QB3P', 'PLNAB6W': 'PLN_AB6W', 'RUBAM3MO': 'RUB_AM3MO', 'MXNMB1T': 'MXN_MB1T', 'BGNAB3SO': 'BGN_AB3SO', 'ISKAM3R': 'ISK_AM3R', 'PKRSM6KI': 'PKR_SM6KI', 'QARAM3Q': 'QAR_AM3Q', 'RONAM3R': 'RON_AM3R', 'SARAM3L': 'SAR_AM3L', 'SEKAB3S': 'SEK_AB3S', 'SGDSB6SO': 'SGD_SB6SO', 'THBSB6TH': 'THB_SB6TH', 'TWDQM3TC': 'TWD_QM3TC', 'TRYAM3T': 'TRY_AM3T', 'USDAM3L': 'USD_AM3L', 'USDSB3L': 'USD_SB3L', 'VNDQM3VN': 'VND_QM3VN', 'ZARQB3ZB': 'ZAR_QB3ZB'}
ois_style = {'EUR': 'OIS_EONIA', 'GBP': 'OIS_SONIA', 'SEK': 'OIS_SEKSTI', 'AUD': 'OIS_AUDRBA', 'CAD': 'OIS_CADCORRA', 'CHF': 'OIS_CHFTOIS', 'DKK': 'OIS_DKKTN',
             'INR': 'OIS_LONGMIOIS', 'JPY': 'OIS_JPYONMU', 'NZD': 'OIS_NZDRBNZ', 'PLN': 'OIS_POLONIA', 'SGD': 'OIS_SGDONDFIX', 'USD': 'OIS_FFER', 'HUF': 'OIS_HUFONIA', 'RUB': 'OIS_RUONIA'}


def get_bh_isin(rics, date):
    field = 'TR.BH.ISIN(SDate = %s)' % date
    df = rd.get_data(rics, field)
    return df


def get_bond_type(is_ilb, cpn):
    inst_type = []
    c_dict = cpn_dict
    for x in range(len(is_ilb)):
        if str(is_ilb[x]).strip() == 'Y':
            inst_type.append('ILB')
        else:
            inst_type.append(c_dict[cpn[x]])
    return inst_type


def get_chain_type(code):
    # 0 : zero
    # 1 : bond
    # 2 : fx fwd
    # 3 : option
    # 4 : futures
    # 5 : ir futures
    # 6 : cds
    # 7 : irs
    # 8 : commodity
    # 9 : depo
    # 10 : fra
    # 11 : ibor fixing
    # 12 : unclassified
    # 13 : fx spot
    # 14 : equity

    if 'Z = ' in code and code[-2:] != 'Z = ':
        classifier = 0
    else:
        data = rd.get_data(Chain(code), 'TR.RIC')
        data.dropna()
        classifier = ''
        r = data['Instrument'].tolist()
        data = rd.get_data(r, 'TR.AssetCategory')
        test_na = data.isna().sum() == len(data)
        if not test_na[1]:
            test = data.groupby('Asset Category Description').size()
            test = pd.DataFrame(test)
            key = test[0].idxmax()
            key = key.lower()

            if 'bond' in key:
                classifier = 1
            elif 'fx forward' in key:
                classifier = 2
            elif 'option' in key:
                classifier = 3
            elif 'future' in key:
                if 'interest' in key:
                    classifier = 5
                else:
                    classifier = 4
            elif 'default' in key:
                classifier = 6
            elif 'swap' in key:
                if 'interest rate' in key or 'overnight index' in key:
                    classifier = 7
                elif 'basis' in key:
                    classifier = 7
            elif 'commodity' in key:
                if 'spot' in key:
                    classifier = 8
                elif 'forward' in key:
                    classifier = 8
            elif 'deposit' in key:
                classifier = 9
            elif 'forward rate' in key:
                classifier = 10
            elif 'ibor' in key or 'fixing' in key:
                classifier = 11
            elif 'share' in key:
                classifier = 14
            else:
                classifier = 12

        else:
            classifier = 12

    return classifier


def unmanaged_chain_resolver(code):
    rics = []
    loop = False
    fields = ['LONGNEXTLR', 'LONGLINK1', 'LONGLINK2', 'LONGLINK3', 'LONGLINK4', 'LONGLINK5', 'LONGLINK6',
              'LONGLINK7', 'LONGLINK8', 'LONGLINK9', 'LONGLINK10', 'LONGLINK11', 'LONGLINK12', 'LONGLINK13', 'LONGLINK14']
    data = rd.get_data(code, fields)
    loop = (data['LONGNEXTLR'].isna())[0] == False
    code_nxt = data['LONGNEXTLR'].values[0] if loop else None
    data = data.drop(columns=['Instrument', 'LONGNEXTLR'])
    data = data.dropna(axis=1)
    rics = data.loc[0].tolist()

    while loop:
        data = rd.get_data(code_nxt, fields)
        loop = (data['LONGNEXTLR'].isna())[0] == False
        code_nxt = data['LONGNEXTLR'].values[0] if loop else None
        data = data.drop(columns=['Instrument', 'LONGNEXTLR'])
        data = data.dropna(axis=1)
        rics += data.loc[0].tolist()

    return rics


def get_swap_styles(rics):
    styles = []
    for n in range(len(rics)):
        if 'OIS' in rics[n] or 'TNS' in rics[n]:
            for m in ois_style:
                if m in rics[n]:
                    styles.append(ois_style[m])
        else:
            for m in irs_style:
                if m in rics[n]:
                    styles.append(irs_style[m])
    return styles


def get_data(ric_chain, at_date='', chain_type=''):
    dict_type = {'zero': 0, 'bond': 1, 'fx forward': 2, 'option': 3, 'futures': 4, 'ir futures': 5, 'cds': 6,
                 'irs': 7, 'commodity': 8, 'deposit': 9, 'fra': 10, 'ibor': 11, 'credit': 12, 'fx spot ric': 13, 'equity': 14}

    if chain_type == '' or chain_type == None:
        chain_type = get_chain_type(ric_chain)
    else:
        chain_type = dict_type[chain_type]

    today_date = datetime.strptime(
        datetime.today().strftime('%Y-%m-%d'), '%Y-%m-%d')
    at_date = datetime.strptime(at_date, '%Y-%m-%d') if at_date != '' else ''
    realtime = True if (today_date == at_date or at_date == '') else False
    crv = pd.DataFrame()

    if chain_type == 0:  # zero
        raw = rd.get_data(Chain(ric_chain), [
                          'MATUR_DATE', 'GV4_TEXT', 'RT_YIELD_1', 'SEC_ACT_1'])
        raw = raw.drop([0, 1, 2])
        raw = raw.dropna()
        raw = raw.reset_index(drop=True)
        total_points = len(raw.index)
        crv['ric'] = raw['Instrument']
        crv['tenor_code'] = raw['GV4_TEXT']
        if realtime:
            crv['yield'] = raw['RT_YIELD_1'] / 100
            crv['maturity'] = raw['MATUR_DATE']
            crv['tenor'] = (pd.to_datetime(
                crv['maturity'], format='%Y-%m-%d') - datetime.now()) / np.timedelta64(1, 'D')
            crv['tenor'] = crv['tenor'] / 365
            crv['discount_factor'] = raw['SEC_ACT_1']

    elif chain_type == 1:  # bonds
        raw = rd.get_data(Chain(ric_chain), [
                          'PRIMACT_1', 'SEC_ACT_1', 'RT_YIELD_1', 'SEC_YLD_1', 'PRC_QL3'])
        raw = raw.dropna()
        raw = raw.reset_index(drop=True)
        crv['ric'] = raw['Instrument']
        tmp_ric = raw['Instrument'].tolist()
        isin = symbol_conversion.Definition(
            symbols=tmp_ric).get_data().data.df
        crv['isin'] = isin['IssueISIN'].tolist()
        meta_data = rd.get_data(tmp_ric, ['TR.FiMaturityDate', 'TR.ADF_BONDSTRUCTURE', 'TR.ADF_RATESTRUCTURE', 'TR.ADF_STRUCTURE', 'TR.FiCouponRate', 'TR.FiCurrency', 'TR.FiDescription',
                                'TR.FiInflationProtected', 'TR.FiCouponType', 'TR.ADF_MARGIN', 'TR.FiSeniorityType', 'TR.FiIsBenchmark', 'TR.UltimateParentId', 'TR.FiIsPutable', 'TR.FiIsCallable', 'TR.FiIsSinkable'])
        meta_data['Ultimate Parent Id'] = meta_data['Ultimate Parent Id'].fillna(
            0)
        meta_data = meta_data[meta_data['Ultimate Parent Id'] != 0]
        meta_data = meta_data.reset_index(drop=True)
        crv['maturity'] = meta_data['Maturity Date']
        crv['coupon'] = meta_data['Coupon Rate'] / 100
        crv['currency'] = meta_data['Currency']
        crv['bond_structure'] = meta_data['Bond Structure']
        crv['rate_structure'] = meta_data['Rate Structure']
        crv['structure'] = meta_data['Structure']
        crv['description'] = meta_data['Description']
        crv['seniority'] = meta_data['Seniority Type']
        crv['bond_type'] = get_bond_type(
            meta_data['Inflation Protected'].tolist(), meta_data['Coupon Type'].tolist())
        crv['quoted_margin'] = pd.to_numeric(meta_data['FRN Margin']) / 100
        crv['quotation_method'] = raw['PRC_QL3']
        crv['issuer_permid'] = meta_data['Ultimate Parent Id']
        crv['putable'] = meta_data['Is Putable']
        crv['callable'] = meta_data['Is Callable']
        crv['sinkable'] = meta_data['Is Sinkable']
        crv['is_benchmark'] = meta_data['Is Benchmark']

        if realtime:
            crv['bid_yield'] = raw['RT_YIELD_1']/100
            crv['ask_yield'] = raw['SEC_YLD_1']/100
            crv['yield'] = crv['bid_yield']
            crv['tenor'] = (pd.to_datetime(
                crv['maturity'], format='%Y-%m-%d') - datetime.now()) / np.timedelta64(1, 'D')
            crv['tenor'] = crv['tenor'] / 365
            crv['bid'] = raw['PRIMACT_1'] / 100
            crv['ask'] = raw['SEC_ACT_1'] / 100

    elif chain_type == 2:  # fx forward
        ric_chain = unmanaged_chain_resolver(ric_chain)
        raw = rd.get_data(ric_chain, ['MATUR_DATE', 'GV4_TEXT', 'BID', 'ASK'])
        raw = raw.dropna(how='all', subset=['BID', 'ASK'])
        raw = raw.reset_index(drop=True)
        crv['ric'] = raw['Instrument']
        crv['tenor_code'] = raw['GV4_TEXT']
        if realtime:
            crv['maturity'] = raw['MATUR_DATE']
            crv['tenor'] = (pd.to_datetime(
                crv['maturity'], format='%Y-%m-%d') - datetime.now()) / np.timedelta64(1, 'D')
            crv['tenor'] = crv['tenor'] / 365
            crv['bid'] = raw['BID']
            crv['ask'] = raw['ASK']

    elif chain_type == 3:  # options
        raw = rd.get_data(Chain(ric_chain), ['EXPIR_DATE', 'BID', 'ASK', 'PRIMACT_1', 'SETTLE',
                          'HST_CLOSE', 'CURRENCY', 'PUTCALLIND', 'GEN_VAL2', 'GEN_VAL3', 'STRIKE_PRC'])
        raw.fillna(value=pd.np.nan, inplace=True)
        raw = raw.dropna(how='all', subset=[
                         'BID', 'ASK', 'PRIMACT_1', 'SETTLE', 'HST_CLOSE', 'STRIKE_PRC', 'GEN_VAL2', 'GEN_VAL3'])
        raw = raw.reset_index(drop=True)
        crv['ric'] = raw['Instrument']
        crv['currency'] = raw['CURRENCY']
        crv['option_type'] = raw['PUTCALLIND']
        if realtime:
            crv['expiry'] = raw['EXPIR_DATE']
            crv['tenor'] = (pd.to_datetime(
                crv['expiry'], format='%Y-%m-%d') - datetime.now()) / np.timedelta64(1, 'D')
            crv['tenor'] = crv['tenor'] / 365
            crv['strike'] = raw['STRIKE_PRC']
            crv['bid'] = raw['BID']
            crv['ask'] = raw['ASK']
            crv['last'] = raw['PRIMACT_1']
            crv['settle'] = raw['SETTLE']
            crv['close'] = raw['HST_CLOSE']
            crv['last_volatility'] = raw['GEN_VAL2'] / 100
            crv['close_volatility'] = raw['GEN_VAL3'] / 100

    elif chain_type == 4 or chain_type == 5:  # futures
        raw = rd.get_data(Chain(ric_chain), [
                          'EXPIR_DATE', 'BID', 'ASK', 'TRDPRC_1', 'SETTLE', 'HST_CLOSE', 'CURRENCY', 'GEN_VAL2'])
        raw.fillna(value=pd.np.nan, inplace=True)
        raw = raw.dropna(how='all', subset=[
                         'BID', 'ASK', 'TRDPRC_1', 'SETTLE', 'HST_CLOSE'])
        raw = raw.reset_index(drop=True)
        crv['ric'] = raw['Instrument']
        crv['currency'] = raw['CURRENCY']
        if realtime:
            denom = 100 if chain_type == 5 else 1
            crv['expiry'] = raw['EXPIR_DATE']
            crv['tenor'] = (pd.to_datetime(
                crv['expiry'], format='%Y-%m-%d') - datetime.now()) / np.timedelta64(1, 'D')
            crv['tenor'] = crv['tenor'] / 365
            crv['bid'] = raw['BID'] if chain_type == 3 else raw['BID'] / denom
            crv['ask'] = raw['ASK'] if chain_type == 3 else raw['ASK'] / denom
            crv['last'] = raw['TRDPRC_1'] if chain_type == 3 else raw['TRDPRC_1'] / denom
            crv['settle'] = raw['SETTLE'] if chain_type == 3 else raw['SETTLE'] / denom
            crv['close'] = raw['HST_CLOSE'] if chain_type == 3 else raw['HST_CLOSE'] / denom
            if chain_type == 5:
                crv['implied_yield'] = raw['GEN_VAL2'] / 100

    elif chain_type == 6:  # cds
        raw = rd.get_data(Chain(ric_chain), [
                          'MATUR_DATE', 'PRIMACT_1', 'BEY_BID', 'GEN_VAL3', 'COUPN_RATE'])
        raw = raw.dropna()
        raw = raw.reset_index(drop=True)
        total_points = len(raw.index)
        tmp_recovery = pd.DataFrame()
        crv['ric'] = raw['Instrument']
        crv['coupon'] = raw['COUPN_RATE']
        crv['upfront'] = raw['BEY_BID']
        crv['recovery'] = raw['GEN_VAL3']/100
        tmp_recovery['tmp'] = crv['recovery'].astype(str)
        tmp_ric = raw['Instrument'].tolist()
        meta_data = rd.get_data(
            tmp_ric, ['TR.CDSPaymentFrequency', 'CURRENCY'])
        crv['cds_structure'] = 'CDSTYPE:AMERCDS CFADJ:YES CLDR:USA_FI DMC:N EMC:S LPAID LTYPE:FLOAT AOD:NO LRECEIVED STEPIN:1 CASHSETTLEDELAY:3WD LTYPE:FIXED CCM:MMA0 FRQ:' + meta_data['Frequency']
        crv['risk_structure'] = 'RISKMODEL:CURVE RECOVERY:' + \
            tmp_recovery['tmp'] + \
            ' IMPROBA:CFI NBDAYS:5 ND:DIS INSTTYPE:CDS IM:LIX'
        crv['currency'] = meta_data['CURRENCY']
        if realtime:
            crv['maturity'] = raw['MATUR_DATE']
            crv['tenor'] = (pd.to_datetime(
                crv['maturity'], format='%Y-%m-%d') - datetime.now()) / np.timedelta64(1, 'D')
            crv['tenor'] = crv['tenor'] / 365
            crv['spread'] = raw['PRIMACT_1']

    elif chain_type == 7:  # irs USDIRS
        ric_chain = unmanaged_chain_resolver(ric_chain)
        raw = rd.get_data(
            ric_chain, ['MATUR_DATE', 'PRIMACT_1', 'SEC_ACT_1', 'CURRENCY'])
        raw = raw.drop([0, 1, 2])
        raw = raw.reset_index(drop=True)
        crv['ric'] = raw['Instrument']
        try:
            crv['style'] = get_swap_styles(crv['ric'].tolist())
        except:
            0
        if realtime:
            crv['maturity'] = raw['MATUR_DATE']
            crv['tenor'] = (pd.to_datetime(
                crv['maturity'], format='%Y-%m-%d') - datetime.now()) / np.timedelta64(1, 'D')
            crv['tenor'] = crv['tenor'] / 365
            crv['pay'] = raw['PRIMACT_1'] / 100
            crv['receive'] = raw['SEC_ACT_1'] / 100

    elif chain_type == 8:  # commodities // e.g. 0#WTC- ENSWAP/INFO
        raw = rd.get_data(Chain(ric_chain), ['MATUR_DATE', 'PRIMACT_1'])
        raw = raw.dropna()
        raw = raw.reset_index(drop=True)

        crv = raw

    elif chain_type == 9:  # depo EURDEPO =
        ric_chain = unmanaged_chain_resolver(ric_chain)
        raw = rd.get_data(ric_chain, ['MATUR_DATE', 'BID', 'ASK'])
        raw = raw.dropna()
        raw = raw.reset_index(drop=True)
        crv['ric'] = raw['Instrument']
        if realtime:
            crv['maturity'] = raw['MATUR_DATE']
            crv['tenor'] = (pd.to_datetime(
                crv['maturity'], format='%Y-%m-%d') - datetime.now()) / np.timedelta64(1, 'D')
            crv['tenor'] = crv['tenor'] / 365
            crv['bid'] = raw['BID'] / 100
            crv['ask'] = raw['ASK'] / 100

    elif chain_type == 10:  # fra EURFRA
        ric_chain = unmanaged_chain_resolver(ric_chain)
        raw = rd.get_data(ric_chain, ['GV4_TEXT', 'BID', 'ASK'])
        raw = raw.dropna(how='all', subset=['BID', 'ASK'])
        raw = raw.reset_index(drop=True)
        crv['ric'] = raw['Instrument']
        crv['notation'] = raw['GV4_TEXT'].str.strip()
        intervals = pd.DataFrame(crv['notation'].str.split(
            'X', 1).tolist(), columns=['start', 'end'])
        intervals = intervals.apply(pd.to_numeric)
        crv['underlying tenor'] = intervals['end'] - intervals['start']
        if realtime:
            t0 = datetime.now()
            for t in range(len(intervals)):
                intervals['start'][t] = t0 + \
                    relativedelta(months=+intervals['start'][t])
                intervals['end'][t] = t0 + \
                    relativedelta(months=+intervals['end'][t])
                intervals['start'][t] = intervals['start'][t].strftime(
                    '%Y-%m-%d')
                intervals['end'][t] = intervals['end'][t].strftime('%Y-%m-%d')
            crv['start'] = intervals['start']
            crv['termination'] = intervals['end']
            crv['tenor'] = (pd.to_datetime(
                crv['termination'], format='%Y-%m-%d') - datetime.now()) / np.timedelta64(1, 'D')
            crv['tenor'] = crv['tenor'] / 365
            crv['bid'] = raw['BID'] / 100
            crv['ask'] = raw['ASK'] / 100

    elif chain_type == 11:  # ibor // USDLIBOR =
        ric_chain = unmanaged_chain_resolver(ric_chain)
        raw = rd.get_data(
            ric_chain, ['GV4_TEXT', 'PRIMACT_1', 'CURRENCY', 'GV2_TEXT'])
        raw.fillna(value=pd.np.nan, inplace=True)
        raw = raw.dropna(how='all', subset=['PRIMACT_1'])
        raw = raw.reset_index(drop=True)
        crv['ric'] = raw['Instrument']
        if sum(raw['CURRENCY'].isna()) == len(raw['CURRENCY']):
            crv['currency'] = raw['GV2_TEXT']
        else:
            crv['currency'] = raw['CURRENCY']
        crv['period'] = raw['GV4_TEXT']
        if realtime:
            crv['yield'] = raw['PRIMACT_1']/100

    elif chain_type == 12:  # credit
        raw = rd.get_data(Chain(ric_chain), ['MATUR_DATE', 'RT_YIELD_1'])
        raw = raw.drop([0, 1, 2])
        raw = raw.dropna()
        raw = raw.reset_index(drop=True)
        crv['ric'] = raw['Instrument']
        if realtime:
            crv['maturity'] = raw['MATUR_DATE']
            crv['tenor'] = (pd.to_datetime(
                crv['maturity'], format='%Y-%m-%d') - datetime.now()) / np.timedelta64(1, 'D')
            crv['tenor'] = crv['tenor'] / 365
            crv['yield'] = raw['RT_YIELD_1']/100

    elif chain_type == 13:  # fx spot or single quote
        raw = rd.get_data(ric_chain, ['PRIMACT_1', 'SEC_ACT_1'])
        raw = raw.dropna(how='all', subset=['PRIMACT_1', 'SEC_ACT_1'])
        raw = raw.reset_index(drop=True)
        crv['ric'] = raw['Instrument']
        if realtime:
            crv['bid'] = raw['PRIMACT_1']
            crv['ask'] = raw['SEC_ACT_1']

    elif chain_type == 14:  # equities
        raw = rd.get_data(Chain(ric_chain), [
                          'DSPLY_NAME', 'TRDPRC_1', 'BID', 'ASK', 'VWAP', 'HST_CLOSE', 'ACVOL_1', 'CURRENCY'])
        raw = raw.dropna(how='all', subset=['HST_CLOSE'])
        raw = raw.reset_index(drop=True)
        crv['ric'] = raw['Instrument']
        if realtime:
            crv['name'] = raw['DSPLY_NAME']
            crv['last'] = raw['TRDPRC_1']
            crv['bid'] = raw['BID']
            crv['ask'] = raw['ASK']
            crv['vwap'] = raw['VWAP']
            crv['close'] = raw['HST_CLOSE']
            crv['acc volume'] = raw['ACVOL_1']
            crv['currency'] = raw['CURRENCY']

    return crv

In [10]:
rd.close_session()
